In [39]:
!pip install tensorflow  opencv-python mediapipe sklearn matplotlib pillow

In [40]:
pip install pysocks scikit-learn  

Note: you may need to restart the kernel to use updated packages.


In [56]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import datetime

In [57]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [58]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 
current_directory = os.getcwd()
# Actions that we try to detect
actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ','ඔ','ඕ','ක්','ග්','ජ්','ට්','ද්','ණ්','ත්','ඩ්','න්','ප්','බ්','ම්','ය්','ර්','ල්','ව්','ස්','හ්'])
length_of_array = len(actions)
print(length_of_array)
# actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ'])	
# Thirty videos worth of data
no_sequences = 30

# Each One Video  going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

30


In [59]:
label_map = {label:num for num, label in enumerate(actions)}

In [60]:
sequences, labels = [], []
# loop each action
for action in actions:
    # Loop through the each action sequence(videos)
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # this array will hold all the frame in one video
        window = []
        # Loop though the each frame 30(Frame)
        for frame_num in range(sequence_length):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))

            if os.path.exists(file_path):
                res = np.load(file_path)
            else:
                print(f"Error: File not found at {file_path}")
            # # using np.load function load the saved np array from folder (path,which action,which video number,which frame number)
            # res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # add loaded framee to window
            window.append(res)
        #    
        sequences.append(window)
        labels.append(label_map[action])

Error: File not found at MP_Data\අ\0\0.npy
Error: File not found at MP_Data\අ\0\1.npy
Error: File not found at MP_Data\අ\0\2.npy
Error: File not found at MP_Data\අ\0\3.npy
Error: File not found at MP_Data\අ\0\4.npy
Error: File not found at MP_Data\අ\0\5.npy
Error: File not found at MP_Data\අ\0\6.npy
Error: File not found at MP_Data\අ\0\7.npy
Error: File not found at MP_Data\අ\0\8.npy
Error: File not found at MP_Data\අ\0\9.npy
Error: File not found at MP_Data\අ\0\10.npy
Error: File not found at MP_Data\අ\0\11.npy
Error: File not found at MP_Data\අ\0\12.npy
Error: File not found at MP_Data\අ\0\13.npy
Error: File not found at MP_Data\අ\0\14.npy
Error: File not found at MP_Data\අ\0\15.npy
Error: File not found at MP_Data\අ\0\16.npy
Error: File not found at MP_Data\අ\0\17.npy
Error: File not found at MP_Data\අ\0\18.npy
Error: File not found at MP_Data\අ\0\19.npy
Error: File not found at MP_Data\අ\0\20.npy
Error: File not found at MP_Data\අ\0\21.npy
Error: File not found at MP_Data\අ\0\22.np

In [51]:
# shape should be 900(30 action 1 action has 30 videos) videos,30 frame each,126(21*3*2) key points in a frame
np.array(sequences).shape

(900, 30, 126)

In [52]:
np.array(labels).shape

(900,)

In [18]:
X = np.array(sequences)

In [19]:
y = to_categorical(labels).astype(int)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
from tensorflow.keras.models import Sequential
# Make us able to action detection
from tensorflow.keras.layers import LSTM, Dense
# Make us able to monitor our modle during taining
from tensorflow.keras.callbacks import TensorBoard

In [21]:
# Get the current date in YYYYMMDD format
current_date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Check if the checkpoint directory exists, if not, create it
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Check if a latest checkpoint exists
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,  # Path to save the model weights
    save_weights_only=True,     # Save only the model weights
    verbose=1,                  # Verbosity mode, 1 for progress bar
    save_freq=50          
)



In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
# load Sequential api
model = Sequential()
# add 3 layers of lstm model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [24]:
# If a latest checkpoint exists, load the weights and continue training
if latest_checkpoint:
    print(f"Loading weights from checkpoint: {latest_checkpoint}")
    model.load_weights(latest_checkpoint)
# else:
#     if os.path.exists('action.h5'):
#         # Load the pre-trained model from the H5 file
#         model = tf.keras.models.load_model('action.h5')

#         # Get the model summary
#         model.summary()

        # # Get the model weights
        # model_weights = loaded_model.get_weights()
        # model = model.set_weights(model_weights)



Loading weights from checkpoint: checkpoints\cp-20231029-124136.ckpt


In [27]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print(f"Test Accuracy: {accuracy:.2f}")

1/1 [==============================] - 1s 917ms/step - loss: 115.2257 - categorical_accuracy: 0.2500
Test Accuracy: 0.25


In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback,cp_callback])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 203525 (795.02 KB)
Trainable params: 20352

In [30]:
res = model.predict(X_test)

1/1 [==============================] - 0s 365ms/step


In [31]:
actions[np.argmax(res[4])]

'උ'

In [32]:
actions[np.argmax(y_test[4])]

'ඌ'

In [33]:
model.save('action.keras')